## Импорт библиотек для анализа данных

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import warnings
from catboost import CatBoostRegressor, Pool, cv

## Отключаем предупреждения

In [3]:
warnings.filterwarnings("ignore")

## Считываем данные и выводим первые 5 строк

In [4]:
df = pd.read_csv("diamonds_train.csv")
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.51,Good,D,SI2,63.9,55.0,1180,5.04,5.10,3.24
1,0.72,Ideal,E,VS2,60.8,57.0,3091,5.79,5.82,3.53
2,0.70,Very Good,D,VVS2,62.8,60.0,4022,5.65,5.69,3.56
3,0.36,Ideal,D,SI1,61.2,57.0,663,4.59,4.63,2.82
4,0.54,Very Good,D,SI1,60.0,59.8,1593,5.30,5.34,3.18


## Выводим основную информацию о датасете

In [5]:
print("Информация о датасете: ")
df.info()

Информация о датасете: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43018 entries, 0 to 43017
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    43018 non-null  float64
 1   cut      43018 non-null  object 
 2   color    43018 non-null  object 
 3   clarity  43018 non-null  object 
 4   depth    43018 non-null  float64
 5   table    43018 non-null  float64
 6   price    43018 non-null  int64  
 7   x        43018 non-null  float64
 8   y        43018 non-null  float64
 9   z        43018 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 3.3+ MB


## Проверем missing values

In [6]:
print("Проверка на отсутствующие значения: ")
df.isnull().sum()

Проверка на отсутствующие значения: 


carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

## Оцениваем статистические показатели

In [7]:
print("Статистические показатели: ")
df.describe()

Статистические показатели: 


,carat,depth,table,price,x,y,z
count,43018.000000,43018.000000,43018.000000,43018.000000,43018.000000,43018.000000,43018.000000
mean,0.796818,61.749672,57.456648,3929.500442,5.730426,5.734358,3.539536
std,0.471362,1.427265,2.238806,3985.321605,1.116451,1.144571,0.703502
min,0.200000,43.000000,43.000000,326.000000,3.730000,3.710000,1.070000
25%,0.400000,61.000000,56.000000,951.000000,4.720000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5317.750000,6.540000,6.530000,4.030000
max,4.500000,79.000000,95.000000,18823.000000,10.230000,58.900000,31.800000


## Выделяем столбцы с признаками и столбец с целевой переменной

In [8]:
x = df.drop("price", axis=1)
y = df["price"]

## Находим категориальные признаки (необходимо для работы CatBoost)

In [9]:
cat_features = x.select_dtypes(include="object").columns.tolist()
print(f"Категориальные признаки: {cat_features}")

Категориальные признаки: ['cut', 'color', 'clarity']


## Разделяем на train и test

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
random_state=42, shuffle=True)

## Задаём параметры модели для дальнейшей их корректировки с помощью кросс-валидации

In [11]:
# params = {
#     "iterations": 500,
#     "learning_rate": 0.1,
#     "depth": 5,
#     "loss_function": "RMSE",
#     "eval_metric": "R2",
#     "random_seed": 42,
#     "verbose": 0
# }

params = {
    "iterations": 1000,
    "learning_rate": 0.15,
    "depth": 5,
    "loss_function": "RMSE",
    "eval_metric": "R2",
    "random_seed": 42,
    "verbose": 0
}

## Проводим кросс-валидацию на 5 folds

In [12]:
print("Кросс-валидация (5 фолдов): ")

train_pool = Pool(x_train, y_train, cat_features=cat_features)

cv_results = cv(
    params=params,
    pool=train_pool,
    fold_count=5,
    shuffle=True,
    seed=42,
    plot=True
)

Кросс-валидация (5 фолдов): 


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.9818096646
bestIteration = 810

Training on fold [1/5]

bestTest = 0.9812889579
bestIteration = 997

Training on fold [2/5]

bestTest = 0.9833497507
bestIteration = 989

Training on fold [3/5]

bestTest = 0.981105863
bestIteration = 786

Training on fold [4/5]

bestTest = 0.9819163365
bestIteration = 999



## Находим, при каких параметрах модель выдавала лучшие показатели

In [13]:
best_iter_idx = cv_results["test-R2-mean"].idxmax()
best_r2 = cv_results.loc[best_iter_idx, "test-R2-mean"]
best_iteration = int(cv_results.loc[best_iter_idx, "iterations"])

## Выводим эти показатели

In [14]:
print(f"Лучшая R²: {best_r2:.4f}")
print(f"Лучшая итерация: {best_iteration}")

Лучшая R²: 0.9819
Лучшая итерация: 985


## Задаём заново параметры для модели, с учётом кросс-валидации и обучаем модель

In [15]:
final_model = CatBoostRegressor(
    iterations=best_iteration,
    learning_rate=params["learning_rate"],
    depth=params["depth"],
    loss_function=params["loss_function"],
    eval_metric=params["eval_metric"],
    random_seed=params["random_seed"],
    verbose=100,
    cat_features=cat_features
)

final_model.fit(x_train, y_train)

0:	learn: 0.2320487	total: 25.9ms	remaining: 25.5s
100:	learn: 0.9788962	total: 2.72s	remaining: 23.8s
200:	learn: 0.9819528	total: 5.28s	remaining: 20.6s
300:	learn: 0.9835527	total: 7.9s	remaining: 18s
400:	learn: 0.9846516	total: 10.4s	remaining: 15.2s
500:	learn: 0.9855871	total: 13.1s	remaining: 12.6s
600:	learn: 0.9865344	total: 15.7s	remaining: 10s
700:	learn: 0.9871927	total: 18.3s	remaining: 7.41s
800:	learn: 0.9877024	total: 21s	remaining: 4.82s
900:	learn: 0.9881401	total: 24.6s	remaining: 2.29s
984:	learn: 0.9884050	total: 26.9s	remaining: 0us


## Делаем итоговое предсказание

In [16]:
y_pred = final_model.predict(x_test)

## Получаем метрики для оценки

In [17]:
test_r2 = r2_score(y_test, y_pred)
test_rmse = mean_squared_error(y_test, y_pred)

## Выводим итоговые результаты

In [18]:
print(f"Итоговые результаты: ")
print(f"R²: {test_r2:.4f}")
print(f"RMSE: {test_rmse:,.2f}")

Итоговые результаты: 
R²: 0.9814
RMSE: 301,000.52


## Сохраняем модель

In [19]:
import joblib

joblib.dump(final_model, "final_model_ivan.pkl")

['final_model_ivan.pkl']

## Считываем файл с тестовыми данными, при необходимости загружаем модель (если программа запускается заново)

In [20]:
df_test = pd.read_csv("diamonds_test.csv")

# final_model = joblib.load("final_model.pkl")

## Выводим первые 5 строк датасета

In [21]:
print("diamonds_test.csv")
df_test.head()

diamonds_test.csv


,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.02,Good,F,SI2,59.2,58.0,6.51,6.56,3.87
1,1,0.70,Very Good,I,VVS1,59.5,58.0,5.78,5.81,3.45
2,2,0.32,Very Good,H,VVS2,63.4,56.0,4.37,4.34,2.76
3,3,0.42,Ideal,F,VVS2,62.2,56.0,4.79,4.82,2.99
4,4,0.40,Ideal,F,VS2,62.3,54.0,4.74,4.77,2.96


## Выводим информацию о тестовом наборе данных

In [22]:
print("Информация о датасете: ")
df.info()

Информация о датасете: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43018 entries, 0 to 43017
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    43018 non-null  float64
 1   cut      43018 non-null  object 
 2   color    43018 non-null  object 
 3   clarity  43018 non-null  object 
 4   depth    43018 non-null  float64
 5   table    43018 non-null  float64
 6   price    43018 non-null  int64  
 7   x        43018 non-null  float64
 8   y        43018 non-null  float64
 9   z        43018 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 3.3+ MB


## Делаем проверку отсутствующих значений в датасете

In [23]:
print("Проверка на отсутствующие значения: ")
df.isnull().sum()

Проверка на отсутствующие значения: 


carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

## Выбираем нужные колонки для предсказания

In [24]:
df_test_features = df_test[x.columns]

## Делаем финальное предсказание

In [25]:
y_submission = final_model.predict(df_test_features)

submission = pd.DataFrame({
        "id": df_test["id"],
        "price": y_submission
    })

## Сохраняем финальное предсказание в файл submission.csv

In [26]:
submission.to_csv("submission_ivab.csv", index=False)
print("Сохранение submission.csv")

Сохранение submission.csv
